# SPOmiAlign Demo: Puck 29 Alignment

**任务**：将 Slide-seq Puck 29 数据配准到 CCF 参考图。
**参数**：
* 旋转：180度

In [ ]:
import os
import sys
import cv2
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../../SPOmiAlign")
from roma import align_and_process_images
from data_preprocessing import rasterize_h5ad_to_image



In [ ]:
# --- 配置 ---
DATA_DIR = "../../SPOmiAlign_Repro"
SAVE_PATH = os.path.join("../../output", "h5ad_2_img", "PUCK29")
os.makedirs(SAVE_PATH, exist_ok=True)

h5ad_path = os.path.join(DATA_DIR, "data_preprocessing", "Puck_Num_29.h5ad")
target_img_path = os.path.join(DATA_DIR, "output_reference", "CCF_100048576_205.png")
gen_img_path = os.path.join(SAVE_PATH, "Gen_img_PUCK29.png")

print("🚀 H5AD 转图像 (180度旋转)...")
_, origin = rasterize_h5ad_to_image(
    input_h5ad=h5ad_path, output_png=gen_img_path,
    x_obs_col="Raw_Slideseq_X", y_obs_col="Raw_Slideseq_Y",
    intensity_obs_col="nFeature_Spatial", intensity_log_transform=True,
    threshold_percentile=80, background="black", point_shape="circle", radius=5,
    enhance=True, rotate=180, scale=1.0,
)

plt.figure(figsize=(6,6))
plt.imshow(cv2.imread(gen_img_path), cmap='gray')
plt.title("Rasterized Puck 29")
plt.axis('off')
plt.show()

In [ ]:
# --- 配准与叠加 ---
print("🚀 执行配准...")
save_align = os.path.join(SAVE_PATH, "alignment")
trans_h5ad = os.path.join(save_align, "transformed.h5ad")
trans_img = os.path.join(save_align, "transformed_h5ad.png")

# 1. 配准
align_and_process_images(
    img1_path=target_img_path, img2_path=gen_img_path, h5ad_path=h5ad_path,
    method='affine+bspline', output_dir=save_align,
    x_obs_col="Raw_Slideseq_X", y_obs_col="Raw_Slideseq_Y",
    rotate=180.0, scale=1.0, origin=origin
)

# 2. 验证图生成
rasterize_h5ad_to_image(
    input_h5ad=trans_h5ad, output_png=trans_img,
    x_obs_col="Raw_Slideseq_X", y_obs_col="Raw_Slideseq_Y",
    intensity_obs_col="nFeature_Spatial", intensity_log_transform=True,
    threshold_percentile=80, background="black", point_shape="circle", radius=2,
    enhance=True, rotate=0.0, scale=1.0,
)

# 3. 叠加显示
t_img = cv2.imread(target_img_path)
s_img = cv2.imread(trans_img, cv2.IMREAD_GRAYSCALE)

if t_img is not None and s_img is not None:
    h, w = s_img.shape
    mask = np.zeros(t_img.shape[:2], dtype=np.uint8)
    mask[:min(mask.shape[0], h), :min(mask.shape[1], w)] = s_img[:min(mask.shape[0], h), :min(mask.shape[1], w)]
    
    overlay = t_img.copy()
    overlay[mask > 50] = [0, 255, 0] # 绿色

    cv2.imwrite(os.path.join(save_align, "final_overlay.png"), overlay)
    
    plt.figure(figsize=(10,10))
    plt.imshow(cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB))
    plt.title("Result: Puck 29 Aligned")
    plt.axis('off')
    plt.show()